<font color=orange>1. Django的初始化配置</font>

In [1]:
# 运行django_setup.py，进行配置。
# 检查settings module是否正确? 我的项目结构里，应该是是config/settings/development.py，
# 检查project_root是否正确?
# 检查INSTALLED_APPS是否正确?
%run django_setup.py


Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'apps.authentication', 'apps.files', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'storages']
Using settings from: config.settings.development
Project root: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend

Installed Apps:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- apps.authentication
- apps.files
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- storages


<font color=orange>2. 测试models.py中模型？</font>

In [ ]:
# 测试是否能成功导入模型？ 
from apps.authentication.models import User, VerificationCode
from django.utils import timezone
from datetime import timedelta
from helpers import queryset_to_dataframe

print("Django环境已成功加载！")

In [3]:
# 测试用户模型 - 创建用户 （使用自定义的create_user方法）
# phone是唯一的必填项
test_user = User.objects.create_user(
    phone='13800138000',
    email='test@example.com',
    password='testpass123'
)
test_superuser = User.objects.create_superuser(
    phone='13800138001',
    email='test@example.com',
    password='testpass123'
)


In [ ]:
from pprint import pprint
pprint(test_user.__dict__)
pprint(test_superuser.__dict__)


In [5]:
# 修改用户参数
test_user.is_staff = True  #这个改完以后，test_user也能够登录系统后台。
test_user.save()

<font color=orange>3.序列化测试</font>




In [ ]:
#test_user.delete()
User.objects.all().delete() #删除所有用户


In [7]:
# 导入序列化器
from apps.authentication.serializers import UserSerializer, UserCreateSerializer

In [8]:
# 测试用户序列化
test_user_data = {
    'phone': '13800138000',
    'password': 'testpass123',
    'confirm_password': 'testpass123',
    'agree_to_terms': True
}

In [ ]:
# 测试创建序列化器 （反序列化，用户输入到后端JSON 到 Python 对象）
serializer = UserCreateSerializer(data=test_user_data)
# 1. 序列化并验证数据
if serializer.is_valid():
    print("数据验证通过")
    print("验证后的数据:", serializer.validated_data)
    
    # 2. 保存数据到数据库， 序列化后需要添加serializer.save()保存数据到数据库
    user = serializer.save()
    print("\n用户创建成功:")
    print(f"ID: {user.id}")
    print(f"Phone: {user.phone}")
    print(f"Role: {user.role}")
    
    # 验证用户是否可以登录
    from django.contrib.auth import authenticate
    auth_user = authenticate(phone=test_user_data['phone'], password=test_user_data['password'])
    if auth_user:
        print("\n用户认证成功!")
    else:
        print("\n用户认证失败!")
else:
    print("验证错误:", serializer.errors)

In [ ]:
# 测试用户对象序列化 （反序列化，后端返回给前端）
user = User.objects.first()  # 获取之前创建的测试用户
user_serializer = UserSerializer(user)
print("\n序列化后的用户数据:")
print(user_serializer.data)

In [ ]:
from pprint import pprint
pprint(user_serializer.data)

<font color=orange>4. 测试services.py</font>

In [ ]:
#test_user.delete()
User.objects.all().delete() #删除所有用户

In [ ]:
from apps.authentication.services import AuthService

# 测试验证码生成
try:
    AuthService.generate_captcha('13800138000', 'register')
    print("验证码生成成功")
except Exception as e:
    print(f"验证码生成失败: {str(e)}")

In [ ]:
# 打印最新的验证码
verification = VerificationCode.objects.filter(phone='13800138000', type='register').last()
print(f"验证码: {verification.code}")


In [ ]:
verifications = VerificationCode.objects.all()
for verification in verifications:
    pprint(f"验证码: {verification}")
    pprint(verification.__dict__)
    print("--------------------------------")

In [ ]:
# 测试用户注册
try:
    result = AuthService.register_user(
        phone='13800138000',
        password='testpass123',
        captcha='669217'  # 使用实际生成的验证码
    )
    print("注册成功:", result)
except Exception as e:
    print(f"注册失败: {str(e)}")


In [ ]:
# 测试密码登录
try:
    result = AuthService.login_with_password('13800138000', 'testpass123')
    print("登录成功:", result)
except Exception as e:
    print(f"登录失败: {str(e)}")

In [ ]:
test_user = User.objects.get(phone='13800138000')
pprint(test_user.__dict__)


In [ ]:
# 3. 生成重置验证码
try:
    AuthService.generate_captcha('13800138000', 'resetPassword')
    print("重置验证码生成成功")
except Exception as e:
    print(f"重置验证码生成失败: {str(e)}")

In [ ]:
# 测试密码重置
try:
    reset_result = AuthService.reset_password(
        phone='13800138000',
        new_password='newpassword123',
        captcha='123456'  # 使用实际生成的重置验证码
    )
    print("密码重置成功:", reset_result)
except Exception as e:
    print(f"密码重置失败: {str(e)}")

In [ ]:
# 打印最新的验证码
verification = VerificationCode.objects.filter(phone='13800138000', type='resetPassword').last()
print(f"验证码: {verification.code}")

In [ ]:
# 4. 重置密码
try:
    reset_result = AuthService.reset_password(
        phone='13800138000',
        new_password='newpassword123',
        captcha='918627'  # 使用实际生成的重置验证码
    )
    print("密码重置成功:", reset_result)
except Exception as e:
    print(f"密码重置失败: {str(e)}")

In [ ]:
# 5. 测试新密码登录
try:
    login_result = AuthService.login_with_password('13800138000', 'newpassword123')
    print("新密码登录成功:", login_result)
except Exception as e:
    print(f"新密码登录失败: {str(e)}")

In [ ]:
# 清除所有验证码
VerificationCode.objects.all().delete()
# 清除所有用户
User.objects.all().delete()


<font color=orange>5. 测试views.py</font>

<font color=red>注意事项：以下测试需确保时间间隔，特别是再测试RegisterView, CaptchaLoginView, PasswordResetView的时候，由于我们验证码发送限制了60秒一次，否则会出错 </font>

In [3]:
# 导入所需模块
from rest_framework.test import APIRequestFactory  ## 使用 DRF 的 APIRequestFactory
from rest_framework.test import force_authenticate
from apps.authentication.views import (
    RegisterView, LoginView, CaptchaRequestView,
    CaptchaLoginView, PasswordResetView, UserProfileView
)
from apps.authentication.models import User, VerificationCode
import json

In [4]:
# 创建请求工厂
# 使用 APIRequestFactory 替代 RequestFactory
factory = APIRequestFactory()

In [20]:
# -----------  请求验证码视图 测试函数   -------------
def test_captcha_request(type):
    print("测试验证码请求:")
    data = {
        'phone': '13800138000',
        'type': type
    }
    request = factory.post(
        '/api/auth/captcha/',
        data,  # 不需要 json.dumps()
        format='json'  # 使用 format='json' 替代 content_type
    )
    
    print('Request:', request)
    print('Request body:', request.body)  # 查看请求体

    response = CaptchaRequestView.as_view()(request)
    print(f"状态码: {response.status_code}")
    print(f"响应: {response.data}\n")
    return response

In [21]:
# -----------  用户注册视图 测试函数   -------------
def test_register():
    print("====\n测试用户注册: notebooks/auth_tests.ipynb=====")
    # 获取最新的验证码
    verification = VerificationCode.objects.filter(
        phone='13800138000',
        type='register'
    ).latest('created_at')

    captcha = str(verification.code)

    data = {
        'phone': '13800138000',
        'password': 'testpass123',
        'confirm_password': 'testpass123',
        'captcha': captcha,
        'agree_to_terms': True
    }
    
    print("\n发送的注册数据:", data)
    print("\n")

    
    request = factory.post(
        '/api/auth/register/',
        data=data,
        format='json'
    )
    
    response = RegisterView.as_view()(request)
    print("====  最终注册响应 notebooks/auth_tests.ipynb=====")
    print(f"\n注册响应状态码: {response.status_code}")
    print(f"注册响应数据: {response.data}")
    return response # 确保验证码是字符串类型

In [ ]:
# -----------  清理测试数据 done test   -------------
print("=== 清理测试数据 ===")
User.objects.filter(phone='13800138000').delete()
VerificationCode.objects.filter(phone='13800138000').delete()

print("\n=== 请求验证码 ===")
captcha_response = test_captcha_request('register')

print("\n=== 执行注册 ===")
register_response = test_register()

In [23]:
# -----------  密码登录视图 测试函数   -------------
def test_password_login():
    print("\n=== 测试密码登录 ===")
    data = {
        'phone_or_email': '13800138000',
        'password': 'testpass123',
        'agree_to_terms': True
    }
    
    print("登录数据:", data)
    
    request = factory.post(
        '/api/auth/login/',
        data=data,
        format='json'
    )
    
    response = LoginView.as_view()(request)
    print(f"登录响应状态码: {response.status_code}")
    print(f"登录响应数据: {response.data}")
    return response

In [ ]:
if register_response.status_code == 201:
    password_login_response = test_password_login()

In [25]:
# -----------  验证码登录视图 测试函数   -------------
def test_captcha_login():
    print("\n=== 测试验证码登录 ===")
    # 1. 先请求验证码
    captcha_response = test_captcha_request('login')
    
    # 2. 获取最新的验证码
    verification = VerificationCode.objects.filter(
        phone='13800138000',
        type='login'
    ).latest('created_at')
    
    captcha = str(verification.code)
    
    # 3. 使用验证码登录
    data = {
        'phone': '13800138000',
        'captcha': captcha,
        'agree_to_terms': True
    }

    
    print("验证码登录数据:", data)
    
    request = factory.post(
        '/api/auth/login/captcha/',
        data=data,
        format='json'
    )
    
    response = CaptchaLoginView.as_view()(request)
    print(f"验证码登录响应状态码: {response.status_code}")
    print(f"验证码登录响应数据: {response.data}")
    return response

In [ ]:
if register_response.status_code == 201:
    # 3. 测试验证码登录
    captcha_login_response = test_captcha_login()

In [29]:
# -----------  密码重置视图 测试函数   -------------
def test_password_reset():
    print("\n=== 测试密码重置 ===")
    # 1. 请求重置密码验证码
    data = {
        'phone': '13800138000',
        'type': 'resetPassword'
    }
    
    request = factory.post(
        '/api/auth/captcha/',
        data=data,
        format='json'
    )
    
    CaptchaRequestView.as_view()(request)
    
    # 2. 获取最新的验证码
    verification = VerificationCode.objects.filter(
        phone='13800138000',
        type='resetPassword'
    ).latest('created_at')
    
    captcha = str(verification.code)
    
    # 3. 重置密码
    reset_data = {
        'phone': '13800138000',
        'new_password': 'newpassword123',
        'confirm_password': 'newpassword123',
        'captcha': captcha
    }
    
    print("密码重置数据:", reset_data)
    
    request = factory.post(
        '/api/auth/password/reset/',
        data=reset_data,
        format='json'
    )
    
    response = PasswordResetView.as_view()(request)
    print(f"密码重置响应状态码: {response.status_code}")
    print(f"密码重置响应数据: {response.data}")
    
    # 4. 使用新密码测试登录
    if response.status_code == 200:
        print("\n=== 测试新密码登录 ===")
        login_data = {
            'phone_or_email': '13800138000',
            'password': 'newpassword123',
            'agree_to_terms': True
        }
        
        request = factory.post(
            '/api/auth/login/',
            data=login_data,
            format='json'
        )
        
        login_response = LoginView.as_view()(request)
        print(f"新密码登录响应状态码: {login_response.status_code}")
        print(f"新密码登录响应数据: {login_response.data}")
    
    return response

In [ ]:
if register_response.status_code == 201:
    # 测试密码重置
    password_reset_response = test_password_reset()

<font color=orange>5. 测试urls.py</font>